In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout,Bidirectional # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

In [ ]:
df = pd.read_csv("/content/train_data.csv")
train_label = df.Label
df2 =pd.read_csv("/content/dev_data.csv")
valid_label = df2.Labels

In [ ]:
# retrive train data
pickle_in = open("/content/clinical_berttrain.pickle", "rb")
clinical_train = pickle.load(pickle_in)
# retrive valid data
pickle_in = open("/content/clinicalbert_valid.pickle", "rb")
clinical_valid = pickle.load(pickle_in)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Ltrain = encoder.fit_transform(train_label)
Lvalid  = encoder.fit_transform(valid_label)

In [ ]:
# from imblearn.over_sampling import SMOTE

# # Assuming 'X' contains the feature data and 'y' contains the corresponding labels

# # Apply SMOTE oversampling
# smote = SMOTE()
# # Assuming X_train and y_train are your training data and label
# clinical_train2, Ltrain2= smote.fit_resample(clinical_train, Ltrain)
# clinical_valid2, Lvalid2= smote.fit_resample(clinical_valid, Lvalid)


In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

# Create an SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)

# Create an OvR classifier using the SVM classifier
ovr_classifier = OneVsRestClassifier(svm_classifier)

# Train the OvR classifier
ovr_classifier.fit(clinical_train,Ltrain)


OneVsRestClassifier(estimator=SVC(kernel='linear', probability=True))

In [ ]:
# classification report
from sklearn.metrics import classification_report
y_pred = ovr_classifier.predict(clinical_valid)
print(classification_report(Lvalid, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.71      0.72      2169
           1       0.40      0.45      0.42       848
           2       0.35      0.23      0.28       228

    accuracy                           0.61      3245
   macro avg       0.49      0.47      0.47      3245
weighted avg       0.61      0.61      0.61      3245



In [ ]:
pickle_in = open("/content/clinical_berttest.pickle", "rb")
test = pickle.load(pickle_in)

In [ ]:
predictions = ovr_classifier.predict(test)

In [ ]:
# Create a DataFrame with the predictions
df_pred = pd.DataFrame({'pred': predictions})

# Save the DataFrame to a CSV file
df_pred.to_csv('clinical-bert(wtout).csv', index=False)